In [1]:
import tweepy
import json
import pandas as pd
import csv

In [2]:
#Twitter API credentials
consumer_key = 'pe7gsS8WNkANobhPvKU5q9PPv'
consumer_secret = 'EF8F0wezQz8HPBRp17DMXTEBrxTXJLBs1mzPnzjLLDlYsHql7D'
access_token = '468248438-aR7mCZ40MKTFq2TNIOGN0IWEegXUBGKDpN3QtDyT'
access_token_secret = 'K5dXjiphLnk17QfWTxigJHOJjgM3005T3c2OZj2X79FA8'

In [4]:
#boundingbox ciudad de Madrid obtenido de https://boundingbox.klokantech.com/
madrid = [-3.7475842804,40.3721683069,-3.6409114868,40.4886258195] 

#para escribir sobre un txt (tomar decision si txt csv o database)
#file =  open('tweets.txt', 'a')

#Definicion palabras claves busqueda
keywords = ['ODS', 'sostenibilidad', 'desarrollo', 'sostenible', 'cooperación']


class MyStreamListener(tweepy.StreamListener):
    
    def on_data(self,data):
        # Twitter returns data in JSON format - we need to decode it first
        try:
            decoded = json.loads(data)
            
        except Exception as e:
            print ("Error on_data: %s" % str(e)) #we don't want the listener to stop
            return True
        
        #En caso de estar geolocalizado guardar la geolocalizacion
        #Si esta geolocalizado dentro de un bounding box (no exacta)
        if decoded.get('place') is not None:
            place_id = decoded.get('place').get('id')
            place_name =decoded.get('place').get('name')
        else:
            place_id = 'None'
            place_name = 'None'
            
        #Si es localizacion exacta
        #Geo is deprecated, they suggest to use simply coordinates
        if decoded.get('cordinates') is not None:
            m_coord = decoded.get('coordinates')
            c=0
            coord=''
            for i in range(0, len(m_coord)-1):
                mc=m_coord[i]
                m_coord=coord+mc+';'#use a different separator!
                c=c+1
            mc=m_coord[c]
            m_coord=coord+mc
        else:
            coord = 'None'
        
        #USER METADATA
        user_name = '@' + decoded.get('user').get('screen_name') #nombre cuenta @itdUPM 
        user_id=decoded.get('user').get('id') #id de la cuenta (int)
        user_loc=decoded.get('user').get('location') 
        user_follow_count=decoded.get('user').get('followers_count')
        user_friends_count=decoded.get('user').get('friends_count')
        user_fav_count=decoded.get('user').get('favourites_count')
        user_status_count=decoded.get('user').get('statuses_count')
        
        #POST METADATA
        created_at = decoded.get('created_at') #Fecha
        text = decoded['text'].replace('\n',' ') #Contenido tweet
        tweet_id = decoded['id'] #tweet id (int64)
        source = decoded['source'] #string source (web client, android, iphone) interesante???
        
        reply_count = decoded['reply_count'] #Number of times this Tweet has been replied to
        retweet_count = decoded['retweet_count'] #Number of times this Tweet has been retweeted
        favorite_count = decoded['favorite_count'] #how many times this Tweet has been liked by Twitter users.
        quote_count = decoded['quote_count']
        
        #REPLY METADATA
        reply_id=decoded['in_reply_to_status_id']
        reply_user_id=decoded['in_reply_to_user_id']
        #in_reply_to_screen_name Si es una respuesta screen_name al que responde
        if decoded['is_quote_status']:
            quote_id=decoded['quoted_status_id']
        else:
            quote_id='None'
        
        #CAREFUL, THIS IS A LIST THAT MAY VARY THE LENGTH
        hashtags_list=decoded.get('entities').get('hashtags')     
        mentions=decoded.get('entities').get('user_mentions')
        
        hashtags=''
        c=0
        if len(hashtags_list)>0:
            for i in range(0, len(hashtags_list)-1):
                mh=hashtags_list[i].get('text')
                hashtags=hashtags+mh+';'
                c=c+1
            mh=hashtags_list[c].get('text')
            hashtags=hashtags+str(mh)
        else:
            hashtags='None'

        mention_ids=''
        c=0
        if len(mentions)>0:
            for i in range(0, len(mentions)-1):
                mid=mentions[i].get('id_str')
                mention_ids=mention_ids+mid+';'#use a different separator!
                c=c+1
            mid=mentions[c].get('id_str')
            mention_ids=mention_ids+str(mid)
        else:
            mentions_ids='None'

        
        #David: I think "," is better separator
        #Check the number of %s!!
        tweet = ['%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s|%s\n' % (tweet_id,
                                    user_name,user_id,created_at,text, 
                                    source, user_loc, user_follow_count, user_friends_count, user_fav_count, 
                                    user_status_count, reply_count,retweet_count,favorite_count,
                                    quote_count, reply_id, reply_user_id,
                                    place_id, place_name, coord, hashtags, mentions )]
        
        # Writing tweets data
        with open('OutputStreamingLocation_2.txt', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(tweet)
        
        return True
        
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

    # returning non-False reconnects the stream, with backoff.

# Writing csv titles
with open('OutputStreamingLocation_2.txt', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['tweet_id','user_name','user_id','created_at','text','source', 'user_loc', 'user_follow',
                                      'user_friends', 'user_fav_count', 'user_status_count',
                                      'reply_count','retweet_count','favorite_count','quote_count', 'reply_id',
                                      'reply_user_id', 'place_id',
                                      'place_name','coord','hashtags','mentions'])
    
if __name__ == '__main__':
    print ('Starting')
    #authorize twitter, initialize tweepy        
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    
    #create the api and the stream object
    myStreamListener = MyStreamListener()
    myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
    myStream.filter(locations = madrid)



Starting
